![Brats official annotations](https://www.med.upenn.edu/cbica/assets/user-content/images/BraTS/brats-tumor-subregions.jpg)

## Problem statement : 

**To predict and localize brain tumors through image segmentation from the MRI dataset availible in Kaggle.**

I'have divided this article into a series of two parts as we are going to train two deep learning models 
for the same dataset but the different tasks.
The model in this parts is a classification model that will detect tumors from the MRI image and then if a tumor
exists, we will further localize the segment of the segment of the brain having a tumor in the next part of this series.


**Prerequisite :**

Deep learning, I'will try to explicate every part thoroughly but in case you find any difficulty, let me know in the comment 
section . Leat's head into the implementation part using python.


# Libraries :

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
import cv2
from skimage import io

import tensorflow as tf
import keras
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers , optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load Data :

In [ ]:
%cd /kaggle/input/healthcare-mri-for-brain-tumor/Healthcare AI Datasets/Brain_MRI

**Convert the CSV file of the dataset into a data frame to perform specific operations on it.**

In [ ]:
data = pd.read_csv('data_mask.csv')
data.head()

* **1. patient_id : Patient id for each record (dtype:Object)**
* **2. image_path : Path to the image or the MRI (dtype:Object)**
* **3. mask_path : Path to the mask of the corresponding image (dtype:Object)**
* **4. mask : Has two values : 0and1 depending on the image of the mask(dtype:int64)**

**View the DataFrame details.**

In [ ]:
data.info()

**count the values of each class**

In [ ]:
data['mask'].value_counts()

**Visualizing the Brain MRI ,corresponding Mask, and MRI with the mask**

In [ ]:
count = 0

fig, axs = plt.subplots(5,3, figsize=(20,50))
for i in range(len(data)):
    if data['mask'][i]==1 and count<5:
        img = io.imread(data.image_path[i])
        axs[count][0].title.set_text('Brain MRI')
        axs[count][0].imshow(img)
        
        mask = io.imread(data.mask_path[i])
        axs[count][1].title.set_text('Mask')
        axs[count][1].imshow(mask, cmap="gray")
        
        img[mask==255] = (255,0,0)
        axs[count][2].title.set_text('MRI + Mask')
        axs[count][2].imshow(img)
        count+=1
        
fig.tight_layout()

**Drop the id as it is not further required for process**

In [ ]:
data = data.drop(columns = ['patient_id'])
data.shape

**Convert the data in the mask column from integer to string format as well require the data in string format.**

In [ ]:
data['mask'] = data['mask'].apply(lambda x: str(x))
data.info()

As you can see, now each feature has the datatype as an object.

**Split the data into train and test sets.**

In [ ]:
train , test = train_test_split(data, test_size=0.15)

 **Augmentation more data using ImageDataGenerator .generates batches of tensor image data with real-time data augmentation.**
**We will create a train_generator and validation_generator from train data and a test_generator from test data.**

In [ ]:
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.15)

train_gen = datagen.flow_from_dataframe(
dataframe=train,
directory='./',
x_col='image_path',
y_col='mask',
subset='training',
batch_size=16,
shuffle=True,
class_mode='categorical',
target_size=(256,256))

valid_gen = datagen.flow_from_dataframe(
dataframe=train,
directory='./',
x_col='image_path',
y_col='mask',
subset='validation',
batch_size=16,
shuffle=True,
class_mode='categorical',
target_size=(256,256))

test_gen = datagen.flow_from_dataframe(
dataframe=test,
directory='./',
x_col='image_path',
y_col='mask',
batch_size=16,
shuffle=True,
target_size=(256,256))

**Now we will learn the concept of Transfer learning and ResNet50 Model which will be used for further training model.**
 
**Transfer Learning as the name suggests , is a technique to use the pre-trained models in your training. You can build your model on top of this pre-trained model. This is a process that helps you decrease the developement time and increase performance.**

**ResNet (Residual Network) is the ANN trained on the ImageNet dataset that can be used to train the model on top of it. ResNet50 is the variant of the resnet model which has 48 convolution layers along with one MaxPool 1 Average Pool Layer.**

**========================================================================================================================================================**

**Here we are using the ResNet50 Model which is a Transfer Learning Model .Using this, we will further add more layers to build our model.**

In [ ]:
BaseModel = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(256,256,3)))
BaseModel.summary()

you can view the layers in the ResNet50 model by using.summary() as shown above

**Freeze the model weights . it means we will keep the weights constant so that it does not update further .**
**This will avoid destroying any information during further training .**

In [ ]:
for layer in BaseModel.layers:
    layers.trainable=False

**Now as stated above we will add more layers on the top of the layers of ResNet50.**
**These layers will learn to turn the old features into predictions on our dataset.**

In [ ]:
headmodel = BaseModel.output
headmodel = AveragePooling2D(pool_size=(4,4))(headmodel)
headmodel = Flatten(name='flatten')(headmodel)
headmodel = Dense(256, activation='relu')(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(256, activation='relu')(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(2, activation='softmax')(headmodel)

model = Model(inputs=BaseModel.input, outputs=headmodel)
model.summary()

these layers are added and you can see them in the summary.

* **1. Pooling layers are used to reduce the dimensions of the feature maps. The Average Pooling layer returns the average of the values.**
* **2. Flatten layers convert our data into a vector**
* **3. Dense layer is the regular deeply connected neural network layer. Bisacally ,it takes input and calculates output=activation(dot(input,kernel)+bias)**
* **4. Dropout layer prevents the model from overfitting . it randomly sets the input units of hidden layers to 0 during training .**

**Compile the above-built model. Compile defines the loss function ,the optimizer and the metrics.**

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

* **Performing early stopping to save the best model with the least validation loss. Early stopping performs a large number of training epochs and stops training once the model performance does not further improve on the velidation dataset.**

* **Model Checkpoint callback is used with training using model.fit() to save the weights at some interval ,so the weights can be loaded later to continue the training from the state saved.**

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
checkpointer = ModelCheckpoint(filepath='/kaggle/working/classifier-resnet-weights.hdf5', verbose=1, save_best_only=True)

**Now ,you train the model and give the callbacks defined above in the parameter.**

In [ ]:
model.fit(train_gen, steps_per_epoch=train_gen.n // 16, epochs=20, validation_data=valid_gen, validation_steps=valid_gen.n // 16, callbacks=[checkpointer, earlystopping])

## if this code was informative for you ,vote it up ,and than kyou